## Face Mask Detection

In [1]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import re

# Defining Model

In [2]:
num_classes = 2
MaskDetectionModel = Sequential()
MaskDetectionModel.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))
MaskDetectionModel.add(Dense(num_classes, activation='softmax'))
MaskDetectionModel.layers[0].trainable = False


# CompileModel

In [3]:
MaskDetectionModel.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])


# Image augmentation for preprocessing image

In [4]:

data_generator_with_aug = ImageDataGenerator(preprocessing_function=preprocess_input,horizontal_flip = True, width_shift_range = 0.1,height_shift_range = 0.1)
            

data_generator_no_aug = ImageDataGenerator(preprocessing_function=preprocess_input)

# Take image directly from director and apply augmentation

In [5]:

image_size = 224


train_generator = data_generator_with_aug.flow_from_directory(
                                        directory='train',
                                        classes = ['with_mask', 'without_mask'],
                                        target_size=(image_size, image_size),
                                        batch_size=10,
                                        class_mode='categorical')

validation_generator = data_generator_no_aug.flow_from_directory(
                                        directory='val',
                                        classes = ['with_mask', 'without_mask'],
                                        target_size=(image_size, image_size),
                                        class_mode='categorical')

Found 8365 images belonging to 2 classes.
Found 1483 images belonging to 2 classes.


# Fit the MaskDetectionModel

In [6]:
fit_stats = MaskDetectionModel.fit_generator(train_generator,
                                       steps_per_epoch=60,
                                       epochs = 4,
                                       validation_data=validation_generator,
                                       validation_steps=1)

C:\Users\vinay\AppData\Local\Temp\ipykernel_21984\591006825.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fit_stats = MaskDetectionModel.fit_generator(train_generator,


Epoch 1/4


60/60 [==============================] - 81s 1s/step - loss: 0.1987 - accuracy: 0.9361 - val_loss: 0.0490 - val_accuracy: 1.0000
Epoch 2/4
60/60 [==============================] - 66s 1s/step - loss: 0.0784 - accuracy: 0.9717 - val_loss: 0.0351 - val_accuracy: 0.9688
Epoch 3/4
60/60 [==============================] - 69s 1s/step - loss: 0.0440 - accuracy: 0.9867 - val_loss: 0.0367 - val_accuracy: 1.0000
Epoch 4/4
60/60 [==============================] - 60s 988ms/step - loss: 0.0349 - accuracy: 0.9933 - val_loss: 0.0728 - val_accuracy: 0.9688


# Testing with test image

In [7]:
test_generator = data_generator_no_aug.flow_from_directory(
    directory='test',
    target_size=(image_size, image_size),
    batch_size=10,
    class_mode=None,
    shuffle=False
)

pred = MaskDetectionModel.predict_generator(test_generator, steps=len(test_generator), verbose=1)
cl = np.round(pred)
filenames = test_generator.filenames

real_class = []
for file in filenames:
    if re.search("with_mask", file):
        real_class.append(1.0)
    else:
        real_class.append(0.0)

predicted_class = cl[:,0]

results = pd.DataFrame({"file": filenames, "pr": pred[:,0], "pred_class": predicted_class, "real_class": real_class})


Found 1200 images belonging to 2 classes.


C:\Users\vinay\AppData\Local\Temp\ipykernel_21984\1164686347.py:9: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  pred = MaskDetectionModel.predict_generator(test_generator, steps=len(test_generator), verbose=1)


120/120 [==============================] - 119s 973ms/step


In [8]:
results

,file,pr,pred_class,real_class
0,with_mask\train_00000859 - Copy (2).jpg,0.998532,1.0,1.0
1,with_mask\train_00000859 - Copy - Copy.jpg,0.998532,1.0,1.0
2,with_mask\train_00000859 - Copy.jpg,0.998532,1.0,1.0
3,with_mask\train_00000859.jpg,0.998532,1.0,1.0
4,with_mask\train_00000860 - Copy (2).jpg,0.999804,1.0,1.0
...,...,...,...,...
1195,without_mask\5_1_3_20170119154358954 - Copy (3...,0.239974,0.0,0.0
1196,without_mask\5_1_3_20170119154358954 - Copy (4...,0.239974,0.0,0.0
1197,without_mask\5_1_3_20170119154358954 - Copy - ...,0.239974,0.0,0.0
1198,without_mask\5_1_3_20170119154358954 - Copy.jpg,0.239974,0.0,0.0


In [9]:
res = sum(1 for x,y in zip(real_class,predicted_class) if x == y) / len(real_class)
print("Accuracy :", res)

Accuracy : 0.98


# Save MaskDetectionModel

In [10]:
MaskDetectionModel.save('maskDetectionModel.h5')

C:\ProgramData\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
